In [3]:
import numpy as np
import pandas as pd

from sklearn import datasets, tree

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [4]:
featuresData = pd.read_csv("../dataset2/kopo_decision_tree_all_new.csv")
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
le_pro = LabelEncoder()
le_holy = LabelEncoder()

featuresData["PRO_ENCO"] = le_pro.fit_transform(featuresData.PROMOTION)
featuresData["HOLY_ENCO"] = le_pro.fit_transform(featuresData.HOLIDAY)

In [7]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]

In [8]:
groupData = featuresData.groupby(groupKey)

In [9]:
len(featuresData[groupKey])

32415

In [10]:
len(featuresData[groupKey].drop_duplicates())

491

In [11]:
len(groupData)

491

In [12]:
tempDF = pd.DataFrame(columns = featuresData.columns)

for i,j in groupData:
    if len(j) > 50:
        tempDF = tempDF.append(j).reset_index(drop=True)

featuresData2=tempDF.reset_index(drop=True)
featuresData2.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_ENCO,HOLY_ENCO
0,A01,PG01,P01,ITEM007,201501,2015,1,64,Y,1,Y,0.230354,1,1
1,A01,PG01,P01,ITEM007,201502,2015,2,53,N,4,Y,0.231741,1,0
2,A01,PG01,P01,ITEM007,201503,2015,3,67,N,4,Y,0.231741,1,0
3,A01,PG01,P01,ITEM007,201504,2015,4,74,Y,2,Y,0.231741,1,1
4,A01,PG01,P01,ITEM007,201505,2015,5,35,N,4,Y,0.231741,1,0


In [13]:
groupData2 = featuresData2.groupby(groupKey)

In [14]:
len(groupData2)

258

In [20]:
def mlFcst(groupData, inputIndex):
    onegroup = groupData.get_group(list(groupData.groups)[inputIndex])
    eachgroup = onegroup.reset_index()
    eachgroup = eachgroup.sort_values(["YEARWEEK"], ascending=[True]).reset_index(drop=True)
    
    if len(onegroup) > 50 :
        # 1. 상관관계를 산출
        corrDf = eachgroup.corr()
        eachgroup = onegroup.reset_index(drop=True)
        eachgroup = eachgroup.sort_values(["YEARWEEK"], ascending=[True]).reset_index(drop=True)

        eachgroup["YEARWEEK"]= eachgroup["YEARWEEK"].astype(int)
        eachgroup["YEAR"]= eachgroup["YEARWEEK"].astype(int)
        eachgroup["WEEK"]= eachgroup["YEARWEEK"].astype(int)
        eachgroup["QTY"]= eachgroup["YEARWEEK"].astype(int)
        eachgroup["PRO_ENCO"]= eachgroup["YEARWEEK"].astype(int)
        eachgroup["HOLY_ENCO"]= eachgroup["YEARWEEK"].astype(int)

        corrDf = eachgroup.corr()
        featureStd = 0.5
        features = corrDf[(corrDf.QTY != 1)\
                      &(abs(corrDf.QTY)>featureStd)].index.tolist()
        label = ["QTY"]
        if features == []:
            return print("------------------선정된 피쳐가 없습니다.------------------")

        # 2. 데이터를 분리한다 features / label
        itemName = set(list(eachgroup.ITEM))
        
        indexStd = round(len(eachgroup)*0.8)
        stdYearweek = eachgroup.loc[indexStd, ["YEARWEEK"]].values[0]

        trainingData_features = (eachgroup.iloc[0:indexStd,:])[features].reset_index(drop=True)
        trainingData_label = (eachgroup.iloc[0:indexStd,:])[label].reset_index(drop=True)
        testData_features = (eachgroup.iloc[indexStd:,:])[features].reset_index(drop=True)
        testData_label = (eachgroup.iloc[indexStd:,:])[label].reset_index(drop=True)

        # 3. 모델선언 및 학습
        from sklearn import tree
        model_method = tree.DecisionTreeRegressor()
        model = model_method.fit(trainingData_features, trainingData_label)
       
        # 4. 예측
        predict = model.predict(testData_features)
        predict = pd.DataFrame(predict).reset_index(drop=True)
        predict.columns = ["PREDICT"]
#         itemName
        testData_label = testData_label.reset_index(drop=True)
        tmpresult = pd.concat( [testData_features,testData_label] , axis=1).reset_index(drop=True)
        result = pd.concat( [tmpresult,predict] , axis=1).reset_index(drop=True)
#         result = tempResult[["PRO_PERCENT","QTY","PREDICT"]]
#         result = tempResult[whatineed_columns]
        print(len(onegroup))
        return result
    else:
        print(len(onegroup))
        print("삐")

In [21]:
for i in range(0,len(groupData2)):
    print(i,"번째")
    mlFcst(groupData2, i)

0 번째
------------------선정된 피쳐가 없습니다.------------------
1 번째
------------------선정된 피쳐가 없습니다.------------------
2 번째
------------------선정된 피쳐가 없습니다.------------------
3 번째
------------------선정된 피쳐가 없습니다.------------------
4 번째
------------------선정된 피쳐가 없습니다.------------------
5 번째
------------------선정된 피쳐가 없습니다.------------------
6 번째
------------------선정된 피쳐가 없습니다.------------------
7 번째
------------------선정된 피쳐가 없습니다.------------------
8 번째
------------------선정된 피쳐가 없습니다.------------------
9 번째
------------------선정된 피쳐가 없습니다.------------------
10 번째
------------------선정된 피쳐가 없습니다.------------------
11 번째
------------------선정된 피쳐가 없습니다.------------------
12 번째
------------------선정된 피쳐가 없습니다.------------------
13 번째
------------------선정된 피쳐가 없습니다.------------------
14 번째
------------------선정된 피쳐가 없습니다.------------------
15 번째
------------------선정된 피쳐가 없습니다.------------------
16 번째
------------------선정된 피쳐가 없습니다.------------------
17 번째
------------------선정된 피쳐가 없습니다.------------------
18

------------------선정된 피쳐가 없습니다.------------------
178 번째
------------------선정된 피쳐가 없습니다.------------------
179 번째
------------------선정된 피쳐가 없습니다.------------------
180 번째
------------------선정된 피쳐가 없습니다.------------------
181 번째
------------------선정된 피쳐가 없습니다.------------------
182 번째
------------------선정된 피쳐가 없습니다.------------------
183 번째
------------------선정된 피쳐가 없습니다.------------------
184 번째
------------------선정된 피쳐가 없습니다.------------------
185 번째
------------------선정된 피쳐가 없습니다.------------------
186 번째
------------------선정된 피쳐가 없습니다.------------------
187 번째
------------------선정된 피쳐가 없습니다.------------------
188 번째
------------------선정된 피쳐가 없습니다.------------------
189 번째
------------------선정된 피쳐가 없습니다.------------------
190 번째
------------------선정된 피쳐가 없습니다.------------------
191 번째
------------------선정된 피쳐가 없습니다.------------------
192 번째
------------------선정된 피쳐가 없습니다.------------------
193 번째
------------------선정된 피쳐가 없습니다.------------------
194 번째
------------------선정된 피쳐가 없습니다.

In [22]:
mlFcst(groupData2, 65)

146


,PRO_PERCENT,QTY,PREDICT
0,0.0,201713,201607.829787
1,0.0,201714,201607.829787
2,0.0,201715,201607.829787
3,0.0,201716,201607.829787
4,0.0,201717,201607.829787
5,0.0,201718,201607.829787
6,0.0,201719,201607.829787
7,0.0,201720,201607.829787
8,0.0,201721,201607.829787
9,0.0,201722,201607.829787
